In [1]:
import json
import numpy as np
import pandas as pd


pd.set_option("display.max_columns", 32)

key = 0
key_size = 8
subkey_size = 2

message_size = 8
submessage_size = 8

In [2]:
message_df = pd.read_csv("message.csv", header=None)

message_block = message_df.to_numpy()

display(message_df)

,0,1,2,3,4,5
0,36,89,93,19,54,45
1,69,75,20,46,35,32
2,72,21,5,108,54,52
3,81,69,48,115,94,8
4,103,24,115,28,6,0
5,11,84,59,104,73,0
6,63,84,57,53,10,0
7,105,2,95,49,19,0


In [3]:
keys_df = pd.read_csv("keys.csv", header=None)

keys_list = [
    keys_df.iloc[i : i + key_size].reset_index(drop=True)
    for i in range(0, len(keys_df), key_size)
]

for key_df in keys_list:
    display(key_df)

,0,1,2,3,4,5,6,7
0,4,7,3,1,4,7,4,7
1,5,9,5,2,5,9,5,9
2,0,0,3,1,0,0,3,1
3,0,0,5,2,0,0,5,2
4,0,0,0,0,4,7,4,7
5,0,0,0,0,5,9,5,9
6,0,0,0,0,0,0,3,1
7,0,0,0,0,0,0,5,2


In [4]:
key_block = keys_list[key].to_numpy()

key_block_inv = np.linalg.inv(key_block).round(0).astype(int)

display(pd.DataFrame(key_block_inv))

,0,1,2,3,4,5,6,7
0,9,-7,-9,7,-9,7,9,-7
1,-5,4,5,-4,5,-4,-5,4
2,0,0,2,-1,0,0,-2,1
3,0,0,-5,3,0,0,5,-3
4,0,0,0,0,9,-7,-2,1
5,0,0,0,0,-5,4,5,-3
6,0,0,0,0,0,0,2,-1
7,0,0,0,0,0,0,-5,3


In [5]:
key_check = np.matmul(key_block, key_block_inv)

display(pd.DataFrame(key_check))

,0,1,2,3,4,5,6,7
0,1,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0
3,0,0,0,1,0,0,0,0
4,0,0,0,0,1,0,0,0
5,0,0,0,0,0,1,0,0
6,0,0,0,0,0,0,1,0
7,0,0,0,0,0,0,0,1


In [6]:
decoded_block = np.matmul(key_block_inv, message_block)

display(pd.DataFrame(decoded_block))

,0,1,2,3,4,5
0,-1258,1684,214,292,827,-231
1,708,-944,-118,-176,-472,131
2,42,-193,-57,44,13,96
3,-117,516,119,-77,5,-236
4,829,-538,603,-533,-458,0
5,-471,630,-339,394,255,0
6,21,166,19,57,1,0
7,0,-414,0,-118,7,0


In [7]:
decoded_block = decoded_block % 118

display(pd.DataFrame(decoded_block))

,0,1,2,3,4,5
0,40,32,96,56,1,5
1,0,0,0,60,0,13
2,42,43,61,44,13,96
3,1,44,1,41,5,0
4,3,52,13,57,14,0
5,1,40,15,40,19,0
6,21,48,19,57,1,0
7,0,58,0,0,7,0


In [8]:
with open("conversion_table.json", encoding="utf-8") as f:
    conversion_table = json.load(f)

print(conversion_table)

{'0': ' ', '1': 'a', '2': 'b', '3': 'c', '4': 'd', '5': 'e', '6': 'f', '7': 'g', '8': 'h', '9': 'i', '10': 'j', '11': 'k', '12': 'l', '13': 'm', '14': 'n', '15': 'o', '16': 'p', '17': 'q', '18': 'r', '19': 's', '20': 't', '21': 'u', '22': 'v', '23': 'w', '24': 'x', '25': 'y', '26': 'z', '27': 'à', '28': 'â', '29': 'á', '30': 'ã', '31': 'ç', '32': 'é', '33': 'ê', '34': 'í', '35': 'ó', '36': 'ô', '37': 'õ', '38': 'ú', '39': 'ü', '40': 'A', '41': 'B', '42': 'C', '43': 'D', '44': 'E', '45': 'F', '46': 'G', '47': 'H', '48': 'I', '49': 'J', '50': 'K', '51': 'L', '52': 'M', '53': 'N', '54': 'O', '55': 'P', '56': 'Q', '57': 'R', '58': 'S', '59': 'T', '60': 'U', '61': 'V', '62': 'W', '63': 'X', '64': 'Y', '65': 'Z', '66': 'Á', '67': 'À', '68': 'Â', '69': 'Ã', '70': 'Ç', '71': 'É', '72': 'Ê', '73': 'Í', '74': 'Ó', '75': 'Ô', '76': 'Õ', '77': 'Ú', '78': 'Ü', '79': '0', '80': '1', '81': '2', '82': '3', '83': '4', '84': '5', '85': '6', '86': '7', '87': '8', '88': '9', '89': ':', '90': ';', '91': '<

In [9]:
decoded_message = decoded_block.flatten()

converted_message = [conversion_table[str(int(c))] for c in decoded_message]

converted_block = np.array(converted_message).reshape(decoded_block.shape)

display(pd.DataFrame(converted_block))

,0,1,2,3,4,5
0,A,é,!,Q,a,e
1,,,,U,,m
2,C,D,V,E,m,!
3,a,E,a,B,e,
4,c,M,m,R,n,
5,a,A,o,A,s,
6,u,I,s,R,a,
7,,S,,,g,
